# Analyse data

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [6]:
det_log = './det/check_nyu'
track_log = './track/logs'
mode_list = ['lowerbound','disco', 'when2com','upperbound']
mode_list_print = ['lowerbound','disco     ', 'when2com  ', 'upperbound']
bootstrap_start = 0
bootstrap_end = 25
rsu = 'no_rsu'

## Detection

In [3]:
def extract_one_det_results(file):
    map_5 = []
    map_7 = []
    with open(file, 'r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            if lines[i][:7] != 'average':
                continue
            split = lines[i].split()
            map_5.append(float(split[4]))
            map_7.append(float(split[10]))
            if len(map_5) >= bootstrap_end:
                break
        f.close()
    map_result = []
    map_result.append(map_5)
    map_result.append(map_7)
    return map_result

def extract__det_results():
    result = []
    for i in range(len(mode_list)):
        file_path = os.path.join(mode_list[i], rsu)
        file_path = os.path.join(file_path, 'log_test.txt')
        file_path = os.path.join(det_log, file_path)
        print(file_path)
        temp_result = extract_one_det_results(file_path)
        result.append(temp_result)
    return result

def compute_mean_variance(det_results):
    result = []
    for i in len(det_results):
        for j in len(det_results[i]):
            temp = []
            mean = np.mean(det_results[i][j])
            std = np.std()

In [4]:
det_results = extract__det_results()
mean = np.mean(det_results, axis = 2)
std = np.std(det_results, axis = 2)

./det/check_nyu/lowerbound/no_rsu/log_test.txt
./det/check_nyu/disco/no_rsu/log_test.txt
./det/check_nyu/when2com/no_rsu/log_test.txt
./det/check_nyu/upperbound/no_rsu/log_test.txt


In [7]:
print("mode          mAP(0.5)          mAP(0.7)")
for i in range(len(mean)):
    st = mode_list_print[i] + "    {0:.3f}".format(mean[i][0]) + "({0:.5f})".format(std[i][0])+ "    {0:.3f}".format(mean[i][1]) + "({0:.5f})".format(std[i][1])
    print(st)

mode          mAP(0.5)          mAP(0.7)
lowerbound    0.497(0.00505)    0.441(0.00303)
disco         0.698(0.00646)    0.643(0.00495)
when2com      0.434(0.01238)    0.393(0.00910)
upperbound    0.707(0.00347)    0.670(0.00246)


In [6]:
for i in range(len(mean)):
    print(mode_list[i], " mAP for 0.5:   ", det_results[i][0])
    print(mode_list[i], " mAP for 0.7:   ", det_results[i][1])

lowerbound  mAP for 0.5:    [0.49900713562965393, 0.4939224421977997, 0.4988515377044678, 0.49761611223220825, 0.49413973093032837, 0.4971027970314026, 0.4889352321624756, 0.4948710799217224, 0.48600688576698303, 0.501207709312439, 0.49157965183258057, 0.49067413806915283, 0.5018049478530884, 0.4974174201488495, 0.5072156190872192, 0.4910263419151306, 0.5050339102745056, 0.5023539066314697, 0.49121975898742676, 0.49950000643730164, 0.4985678493976593, 0.49659666419029236, 0.49934840202331543, 0.49260687828063965, 0.500501275062561]
lowerbound  mAP for 0.7:    [0.4421604871749878, 0.44027259945869446, 0.4383939504623413, 0.44108614325523376, 0.4399482011795044, 0.4382987916469574, 0.4359024167060852, 0.4399508833885193, 0.4344748258590698, 0.4429028630256653, 0.4399273693561554, 0.4362139403820038, 0.44486764073371887, 0.44215914607048035, 0.4484386146068573, 0.43725186586380005, 0.4426913857460022, 0.44167232513427734, 0.4382147192955017, 0.44344666600227356, 0.4422401189804077, 0.4389

## Tracking

In [8]:
metric_list = ['bootstrap', 'MOTA', 'MOTP', 'HOTA', 'DetA', 'AssA', 'DetRe', 'DetPr', 'AssRe', 'AssPr', 'LocA']
def extract_track_result():
    result = []
    for i in range(len(mode_list)):
        file_path = os.path.join(track_log, mode_list[i])
        file_path = os.path.join(file_path, "logs_no_rsu_metric_all.npy")
        print(file_path)
        data = np.load(file_path, allow_pickle=True)
        result.append(data.item()['mean'][bootstrap_start:bootstrap_end])
    return result

In [10]:
track_results = extract_track_result()

./track/logs/lowerbound/logs_no_rsu_metric_all.npy
./track/logs/disco/logs_no_rsu_metric_all.npy
./track/logs/when2com/logs_no_rsu_metric_all.npy
./track/logs/upperbound/logs_no_rsu_metric_all.npy


In [11]:
track_mean = np.mean(track_results, axis = 1)
track_std = np.std(track_results, axis = 1)

In [12]:
def print_metric(start, end):
    title = "mode"
    for i in range(start, end):
        title += "           " + metric_list[i]
    print(title)
    for i in range(len(mean)):
        st = mode_list_print[i]
        for j in range(start, end):
            st += "  {0:.3f}".format(track_mean[i][j]) + "({0:.3f})".format(track_std[i][j])
        print(st)

In [13]:
print_metric(1,6)
print_metric(6,len(metric_list))

mode           MOTA           MOTP           HOTA           DetA           AssA
lowerbound  35.515(0.504)  84.125(0.268)  34.279(0.246)  33.548(0.348)  36.342(0.246)
disco       56.090(0.662)  86.182(0.160)  44.471(0.277)  51.802(0.499)  39.204(0.188)
when2com    29.203(0.849)  85.682(0.382)  30.750(0.458)  27.519(0.840)  35.391(0.263)
upperbound  58.119(0.414)  85.462(0.151)  44.789(0.178)  53.029(0.296)  38.855(0.158)
mode           DetRe           DetPr           AssRe           AssPr           LocA
lowerbound  34.984(0.391)  82.410(0.370)  46.895(0.316)  58.683(0.208)  86.342(0.177)
disco       54.189(0.540)  86.300(0.569)  50.795(0.224)  59.061(0.121)  88.033(0.119)
when2com    28.304(0.912)  85.662(0.613)  46.410(0.264)  59.271(0.387)  87.748(0.301)
upperbound  55.180(0.281)  86.331(0.275)  50.221(0.187)  58.614(0.150)  87.329(0.114)
